# Prophet Walkforward Analysis with Economic Indicators

기존 바이럴 지수에 **경제 지표(kospi, usd_krw, brent_oil)**를 추가 외생변수로 포함

## 비교 분석
1. 기존: viral_index만 사용
2. 신규: viral_index + 경제지표 사용

In [1]:
import os
import pandas as pd
import numpy as np
from prophet import Prophet
from supabase import create_client
from dotenv import load_dotenv
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import logging
import warnings

warnings.filterwarnings('ignore')
logging.getLogger('prophet').setLevel(logging.ERROR)
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)

load_dotenv('/Users/minzzy/Desktop/statrack/book-review-analysis/.env')
supabase = create_client(os.getenv('SUPABASE_URL'), os.getenv('SUPABASE_KEY'))

DECAY_RATE = 0.5

## 1. 데이터 로드

In [2]:
print("="*60)
print("[Prophet with Economic Indicators]")
print("="*60)

# 바이럴 지수
viral_df = pd.read_csv('../weekly_news_viral_index_revised.csv')
viral_df['ymw'] = viral_df['ymw'].astype(str)
print(f"Viral Index: {len(viral_df)}개")

# 경제지표 (DB에서)
print("\n경제지표 로드 중...")
all_econ = []
offset = 0
while True:
    res = supabase.table("books_with_tags_variables").select("ymw, kospi, usd_krw, brent_oil").range(offset, offset + 999).execute()
    if not res.data:
        break
    all_econ.extend(res.data)
    offset += 1000

df_econ = pd.DataFrame(all_econ)
df_econ['ymw'] = df_econ['ymw'].astype(str)
df_econ = df_econ.drop_duplicates(subset=['ymw'])
print(f"경제지표: {len(df_econ)}개 주차")
print(df_econ.head())

# 베스트셀러 데이터
bs_res = supabase.table('weekly_bestsellers').select('ymw, rank, product_code').execute()
books_res = supabase.table('books').select('product_code, category_1').execute()

df_bs = pd.DataFrame(bs_res.data)
df_bs['ymw'] = df_bs['ymw'].astype(str)
df_books = pd.DataFrame(books_res.data)

print(f"\nBestsellers: {len(df_bs)}개")
print(f"Books: {len(df_books)}개")

[Prophet with Economic Indicators]
Viral Index: 550개

경제지표 로드 중...
경제지표: 51개 주차
       ymw    kospi  usd_krw  brent_oil
0  2025012  2497.40  1458.89      79.92
1  2025013  2518.03  1435.53      79.29
2  2025014  2536.80  1417.92      77.49
3  2025021  2481.69  1459.05      76.20
4  2025022  2539.05  1453.21      77.00

Bestsellers: 1000개
Books: 164개


## 2. Decay Score 계산

In [3]:
def calculate_decay_scores(df_bs, df_books):
    book_cat = df_books.set_index('product_code')['category_1'].to_dict()
    all_ymw = sorted(df_bs['ymw'].unique())
    all_books = df_bs['product_code'].unique()
    
    book_week_score = {}
    for _, row in df_bs.iterrows():
        ymw, pc, rank = row['ymw'], row['product_code'], row['rank']
        score = 21 - rank
        if pc not in book_week_score:
            book_week_score[pc] = {}
        book_week_score[pc][ymw] = score
    
    decay_scores = []
    for pc in all_books:
        cat = book_cat.get(pc, '미분류')
        prev_score = 0
        for ymw in all_ymw:
            current_raw = book_week_score.get(pc, {}).get(ymw, 0)
            if current_raw > 0:
                final_score = current_raw
            else:
                final_score = prev_score * DECAY_RATE
            if final_score < 0.1:
                final_score = 0
            if final_score > 0:
                decay_scores.append({'ymw': ymw, 'product_code': pc, 'category_1': cat, 'decay_score': final_score})
            prev_score = final_score
    
    df_decay = pd.DataFrame(decay_scores)
    bs_weighted = df_decay.groupby(['ymw', 'category_1'])['decay_score'].sum().unstack(fill_value=0)
    return bs_weighted

bs_weighted = calculate_decay_scores(df_bs, df_books)
print(f"Decay Score: {bs_weighted.shape[0]}개 주차, {bs_weighted.shape[1]}개 카테고리")

Decay Score: 51개 주차, 10개 카테고리


## 3. Prophet 분석 (바이럴 지수만 vs 바이럴+경제지표)

In [4]:
categories = [c for c in bs_weighted.columns if c and c != '미분류']
lags = [0, 1, 2, 3, 4]
min_train = 5

results_viral_only = []
results_viral_econ = []

print("\n" + "="*70)
print("Prophet Walkforward 분석 (바이럴만 vs 바이럴+경제지표)")
print("="*70)

for cat in categories:
    print(f"\n📌 {cat}")
    
    best_viral = {'mae': float('inf'), 'lag': 0}
    best_econ = {'mae': float('inf'), 'lag': 0}
    
    for lag in lags:
        # 데이터 준비
        cat_viral = viral_df[viral_df['category'] == cat].copy()
        if cat_viral.empty:
            continue
        
        cat_bs = bs_weighted[cat].reset_index()
        cat_bs.columns = ['ymw', 'y']
        cat_bs['ymw'] = cat_bs['ymw'].astype(str)
        
        # 바이럴 + 경제지표 병합
        df_model = pd.merge(cat_viral, cat_bs, on='ymw', how='inner')
        df_model = pd.merge(df_model, df_econ, on='ymw', how='left')
        df_model = df_model.rename(columns={'end_date': 'ds'})
        df_model['ds'] = pd.to_datetime(df_model['ds'])
        df_model = df_model.sort_values('ds').reset_index(drop=True)
        
        # Lag 적용
        df_model['viral_lag'] = df_model['viral_index_smoothed'].shift(lag)
        df_model['kospi_lag'] = df_model['kospi'].shift(lag)
        df_model['usd_lag'] = df_model['usd_krw'].shift(lag)
        df_model['oil_lag'] = df_model['brent_oil'].shift(lag)
        
        df_wf = df_model[['ds', 'ymw', 'y', 'viral_lag', 'kospi_lag', 'usd_lag', 'oil_lag']].dropna().reset_index(drop=True)
        
        if len(df_wf) <= min_train:
            continue
        
        # 정규화
        scaler = StandardScaler()
        df_wf[['viral_norm', 'kospi_norm', 'usd_norm', 'oil_norm']] = scaler.fit_transform(
            df_wf[['viral_lag', 'kospi_lag', 'usd_lag', 'oil_lag']])
        
        # === 1. 바이럴만 ===
        preds_viral = []
        for i in range(min_train, len(df_wf)):
            train = df_wf.iloc[:i]
            test = df_wf.iloc[[i]]
            try:
                m = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
                m.add_regressor('viral_norm')
                m.fit(train[['ds', 'y', 'viral_norm']])
                forecast = m.predict(test[['ds', 'viral_norm']])
                preds_viral.append({'actual': test['y'].values[0], 'pred': max(0, forecast['yhat'].values[0])})
            except:
                continue
        
        if preds_viral:
            mae_v = mean_absolute_error([p['actual'] for p in preds_viral], [p['pred'] for p in preds_viral])
            if mae_v < best_viral['mae']:
                best_viral = {'mae': mae_v, 'lag': lag, 'preds': preds_viral}
        
        # === 2. 바이럴 + 경제지표 ===
        preds_econ = []
        for i in range(min_train, len(df_wf)):
            train = df_wf.iloc[:i]
            test = df_wf.iloc[[i]]
            try:
                m = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
                m.add_regressor('viral_norm')
                m.add_regressor('kospi_norm')
                m.add_regressor('usd_norm')
                m.add_regressor('oil_norm')
                m.fit(train[['ds', 'y', 'viral_norm', 'kospi_norm', 'usd_norm', 'oil_norm']])
                forecast = m.predict(test[['ds', 'viral_norm', 'kospi_norm', 'usd_norm', 'oil_norm']])
                preds_econ.append({'actual': test['y'].values[0], 'pred': max(0, forecast['yhat'].values[0])})
            except:
                continue
        
        if preds_econ:
            mae_e = mean_absolute_error([p['actual'] for p in preds_econ], [p['pred'] for p in preds_econ])
            if mae_e < best_econ['mae']:
                best_econ = {'mae': mae_e, 'lag': lag, 'preds': preds_econ}
    
    if best_viral['mae'] < float('inf'):
        rmse_v = np.sqrt(mean_squared_error([p['actual'] for p in best_viral['preds']], [p['pred'] for p in best_viral['preds']]))
        results_viral_only.append({'category': cat, 'lag': best_viral['lag'], 'mae': best_viral['mae'], 'rmse': rmse_v})
    
    if best_econ['mae'] < float('inf'):
        rmse_e = np.sqrt(mean_squared_error([p['actual'] for p in best_econ['preds']], [p['pred'] for p in best_econ['preds']]))
        results_viral_econ.append({'category': cat, 'lag': best_econ['lag'], 'mae': best_econ['mae'], 'rmse': rmse_e})
    
    if best_viral['mae'] < float('inf') and best_econ['mae'] < float('inf'):
        diff = best_econ['mae'] - best_viral['mae']
        sign = '+' if diff > 0 else ''
        print(f"  바이럴만: MAE={best_viral['mae']:.2f} (lag={best_viral['lag']})")
        print(f"  +경제지표: MAE={best_econ['mae']:.2f} (lag={best_econ['lag']}) [{sign}{diff:.2f}]")

10:20:02 - cmdstanpy - INFO - Chain [1] start processing
10:20:02 - cmdstanpy - INFO - Chain [1] done processing



Prophet Walkforward 분석 (바이럴만 vs 바이럴+경제지표)

📌 거시경제/금융정책


10:20:02 - cmdstanpy - INFO - Chain [1] start processing
10:20:02 - cmdstanpy - INFO - Chain [1] done processing
10:20:02 - cmdstanpy - INFO - Chain [1] start processing
10:20:02 - cmdstanpy - INFO - Chain [1] done processing
10:20:02 - cmdstanpy - INFO - Chain [1] start processing
10:20:02 - cmdstanpy - INFO - Chain [1] done processing
10:20:02 - cmdstanpy - INFO - Chain [1] start processing
10:20:02 - cmdstanpy - INFO - Chain [1] done processing
10:20:02 - cmdstanpy - INFO - Chain [1] start processing
10:20:02 - cmdstanpy - INFO - Chain [1] done processing
10:20:02 - cmdstanpy - INFO - Chain [1] start processing
10:20:02 - cmdstanpy - INFO - Chain [1] done processing
10:20:02 - cmdstanpy - INFO - Chain [1] start processing
10:20:02 - cmdstanpy - INFO - Chain [1] done processing
10:20:02 - cmdstanpy - INFO - Chain [1] start processing
10:20:02 - cmdstanpy - INFO - Chain [1] done processing
10:20:02 - cmdstanpy - INFO - Chain [1] start processing
10:20:02 - cmdstanpy - INFO - Chain [1]

  바이럴만: MAE=10.35 (lag=3)
  +경제지표: MAE=9.35 (lag=2) [-0.99]

📌 경제이론/학술


10:20:55 - cmdstanpy - INFO - Chain [1] start processing
10:20:55 - cmdstanpy - INFO - Chain [1] done processing
10:20:55 - cmdstanpy - INFO - Chain [1] start processing
10:20:55 - cmdstanpy - INFO - Chain [1] done processing
10:20:55 - cmdstanpy - INFO - Chain [1] start processing
10:20:55 - cmdstanpy - INFO - Chain [1] done processing
10:20:56 - cmdstanpy - INFO - Chain [1] start processing
10:20:56 - cmdstanpy - INFO - Chain [1] done processing
10:20:56 - cmdstanpy - INFO - Chain [1] start processing
10:20:56 - cmdstanpy - INFO - Chain [1] done processing
10:20:56 - cmdstanpy - INFO - Chain [1] start processing
10:20:56 - cmdstanpy - INFO - Chain [1] done processing
10:20:56 - cmdstanpy - INFO - Chain [1] start processing
10:20:56 - cmdstanpy - INFO - Chain [1] done processing
10:20:56 - cmdstanpy - INFO - Chain [1] start processing
10:20:56 - cmdstanpy - INFO - Chain [1] done processing
10:20:56 - cmdstanpy - INFO - Chain [1] start processing
10:20:56 - cmdstanpy - INFO - Chain [1]

...

  바이럴만: MAE=6.91 (lag=2)
  +경제지표: MAE=7.37 (lag=3) [+0.46]

📌 테크/스타트업


10:26:39 - cmdstanpy - INFO - Chain [1] start processing
10:26:39 - cmdstanpy - INFO - Chain [1] done processing
10:26:39 - cmdstanpy - INFO - Chain [1] start processing
10:26:39 - cmdstanpy - INFO - Chain [1] done processing
10:26:39 - cmdstanpy - INFO - Chain [1] start processing
10:26:39 - cmdstanpy - INFO - Chain [1] done processing
10:26:39 - cmdstanpy - INFO - Chain [1] start processing
10:26:39 - cmdstanpy - INFO - Chain [1] done processing
10:26:39 - cmdstanpy - INFO - Chain [1] start processing
10:26:39 - cmdstanpy - INFO - Chain [1] done processing
10:26:39 - cmdstanpy - INFO - Chain [1] start processing
10:26:39 - cmdstanpy - INFO - Chain [1] done processing
10:26:39 - cmdstanpy - INFO - Chain [1] start processing
10:26:39 - cmdstanpy - INFO - Chain [1] done processing
10:26:39 - cmdstanpy - INFO - Chain [1] start processing
10:26:40 - cmdstanpy - INFO - Chain [1] done processing
10:26:40 - cmdstanpy - INFO - Chain [1] start processing
10:26:40 - cmdstanpy - INFO - Chain [1]

  바이럴만: MAE=6.28 (lag=4)
  +경제지표: MAE=7.47 (lag=0) [+1.18]

📌 투자철학/대가


10:27:24 - cmdstanpy - INFO - Chain [1] start processing
10:27:24 - cmdstanpy - INFO - Chain [1] done processing
10:27:24 - cmdstanpy - INFO - Chain [1] start processing
10:27:24 - cmdstanpy - INFO - Chain [1] done processing
10:27:24 - cmdstanpy - INFO - Chain [1] start processing
10:27:24 - cmdstanpy - INFO - Chain [1] done processing
10:27:24 - cmdstanpy - INFO - Chain [1] start processing
10:27:24 - cmdstanpy - INFO - Chain [1] done processing
10:27:24 - cmdstanpy - INFO - Chain [1] start processing
10:27:24 - cmdstanpy - INFO - Chain [1] done processing
10:27:24 - cmdstanpy - INFO - Chain [1] start processing
10:27:24 - cmdstanpy - INFO - Chain [1] done processing
10:27:24 - cmdstanpy - INFO - Chain [1] start processing
10:27:24 - cmdstanpy - INFO - Chain [1] done processing
10:27:24 - cmdstanpy - INFO - Chain [1] start processing
10:27:24 - cmdstanpy - INFO - Chain [1] done processing
10:27:24 - cmdstanpy - INFO - Chain [1] start processing
10:27:24 - cmdstanpy - INFO - Chain [1]

  바이럴만: MAE=8.38 (lag=3)
  +경제지표: MAE=6.11 (lag=3) [-2.27]


## 4. 결과 비교

In [5]:
df_v = pd.DataFrame(results_viral_only)
df_e = pd.DataFrame(results_viral_econ)

print("\n" + "="*70)
print("결과 비교: 바이럴만 vs 바이럴+경제지표")
print("="*70)

comparison = pd.merge(df_v, df_e, on='category', suffixes=('_viral', '_econ'))
comparison['mae_diff'] = comparison['mae_econ'] - comparison['mae_viral']
comparison['mae_diff_pct'] = (comparison['mae_diff'] / comparison['mae_viral']) * 100

print(f"\n{'카테고리':<18} {'MAE(바이럴)':<12} {'MAE(+경제)':<12} {'차이':<10} {'변화율':<10}")
print("-"*62)
for _, row in comparison.sort_values('mae_diff').iterrows():
    sign = '+' if row['mae_diff'] > 0 else ''
    print(f"{row['category']:<18} {row['mae_viral']:<12.2f} {row['mae_econ']:<12.2f} {sign}{row['mae_diff']:<9.2f} {sign}{row['mae_diff_pct']:.1f}%")

print("-"*62)
print(f"{'평균':<18} {comparison['mae_viral'].mean():<12.2f} {comparison['mae_econ'].mean():<12.2f} {comparison['mae_diff'].mean():<9.2f} {comparison['mae_diff_pct'].mean():.1f}%")

# 개선/악화 카운트
improved = (comparison['mae_diff'] < 0).sum()
worsened = (comparison['mae_diff'] > 0).sum()
print(f"\n개선된 카테고리: {improved}개")
print(f"악화된 카테고리: {worsened}개")


결과 비교: 바이럴만 vs 바이럴+경제지표

카테고리               MAE(바이럴)     MAE(+경제)     차이         변화율       
--------------------------------------------------------------
기업경영/리더십           11.70        8.73         -2.97     -25.4%
투자철학/대가            8.38         6.11         -2.27     -27.1%
거시경제/금융정책          10.35        9.35         -0.99     -9.6%
경제이론/학술            4.69         4.06         -0.63     -13.4%
금융시스템/위기           7.62         7.99         +0.37      +4.9%
지정학/국제정세           6.91         7.37         +0.46      +6.6%
재테크/개인금융           8.31         8.82         +0.51      +6.2%
주식투자/트레이딩          10.56        11.46        +0.90      +8.5%
테크/스타트업            6.28         7.47         +1.18      +18.9%
부동산/실물자산           6.80         8.46         +1.66      +24.5%
--------------------------------------------------------------
평균                 8.16         7.98         -0.18     -0.6%

개선된 카테고리: 4개
악화된 카테고리: 6개


## 5. 결론

In [6]:
print("\n" + "="*70)
print("결론")
print("="*70)

avg_viral = comparison['mae_viral'].mean()
avg_econ = comparison['mae_econ'].mean()
avg_diff_pct = comparison['mae_diff_pct'].mean()

if avg_diff_pct < -5:
    conclusion = "경제지표 추가로 예측 성능이 유의미하게 개선됨"
elif avg_diff_pct > 5:
    conclusion = "경제지표 추가로 예측 성능이 오히려 악화됨 (과적합 가능성)"
else:
    conclusion = "경제지표 추가 효과가 미미함"

print(f"""
1. 바이럴 지수만 사용 시 평균 MAE: {avg_viral:.2f}
2. 바이럴 + 경제지표 사용 시 평균 MAE: {avg_econ:.2f}
3. 평균 변화율: {avg_diff_pct:+.1f}%

결론: {conclusion}
""")


결론

1. 바이럴 지수만 사용 시 평균 MAE: 8.16
2. 바이럴 + 경제지표 사용 시 평균 MAE: 7.98
3. 평균 변화율: -0.6%

결론: 경제지표 추가 효과가 미미함

